In [65]:
from wordfreq import word_frequency
from googletrans import Translator
import editdistance
from unidecode import unidecode
import time

translator = Translator()

This script will process the data files to add information about translations and frequencies.

In [70]:
lang='es_en'
fn = 'data/data_{0}/{0}.slam.20171218.train.new'.format(lang)
(src, dest)=lang.split('_')
print("Translating %s to %s" %(src,dest))

Translating es to en


In [71]:
fn

'data/data_es_en/es_en.slam.20171218.train.new'

In [24]:
#Todd's code
# trans_cache = {}

# newlines=[]
# with open(fn, 'r') as f:
#     for line in f:
#         if len(line)>1:
#             #print("***"+line)
#             if line[0]!='#':
#                 # process and translate
#                 pieces = line.split()
#                 idx = pieces[0]
#                 word = pieces[1]
#                 root_token = pieces[2]
#                 part_of_speech = pieces[3]
#                 morph_feats = pieces[4]
#                 dependency_label = pieces[5]
#                 dependency_edge_head = pieces[6]
                
#                 clean_word = word.lower()
#                 if '\'' not in clean_word:
#                     if clean_word not in trans_cache:
#                         translation_obj = translator.translate(clean_word,src=src,dest=dest)
#                         translation = translation_obj.text
#                         print("%s to %s" % (clean_word, translation))  # need to do some clean up of the translation
#                         trans_cache[clean_word]=translation
#                     else:
#                         translation = trans_cache[clean_word]
                
#                 trans_freq = word_frequency(translation, dest)
#                 src_frec = word_frequency(word, src)
                
                
#                 if len(pieces) == 8:
#                     label = pieces[7]
#                     repl = [idx, word, root_token, part_of_speech, morph_feats, dependency_label, dependency_edge_head, label]
#                 else:
#                     repl = [idx, word, root_token, part_of_speech, morph_feats, dependency_label, dependency_edge_head]
#                 newlines.append("\t".join(repl)+'\n')
#             else:
#                 newlines.append(line)
#         else:
#             newlines.append(line)

# print(len(newlines))
# with open('test.txt','w') as fp:
#     fp.writelines(newlines)

i to yo
am to a.m
a to un
boy to chico
from to de
mexico to Méjico
my to mi
name to nombre
is to es
pedro to pedro
she to ella
girl to niña
he to él
need to necesitar
taxi to taxi
where to dónde
have to tener
reservation to reserva
fine to multa
when to cuando
newspaper to periódico
stop to detener
now to ahora
room to habitación
today to hoy
woman to mujer
man to hombre
child to niño
good to bueno
morning to Mañana
and to y
night to noche
hello to Hola
thanks to Gracias
how to cómo
are to son
you to tú
please to Por favor
sorry to lo siento
andrea to andrea
bye to adiós
table to mesa
welcome to Bienvenido
to to a
yes to sí
excuse to excusa
me to yo
drinking to bebida
water to agua
eat to comer
more to Más
check to comprobar
what to qué
day to día
boys to chicos
live to vivir
in to en
mother to madre
father to padre
girls to chicas
brother to hermano
husband to marido
one to uno
two to dos
son to hijo
dad to papá
can to poder
help to ayuda
it to eso
there to ahí
not to no
going to yend

prison to prisión
right to derecho
palace to palacio
square to cuadrado
zoo to zoo
west to Oeste
north to norte
windows to ventanas
radio to radio
television to televisión
checkout to revisa
wallet to billetera
razor to maquinilla de afeitar
roof to techo
cabinet to gabinete
glasses to gafas
leaf to hoja
engine to motor
machine to máquina
object to objeto
powder to polvo
pair to par
knife to cuchillo
these to estas
all to todas
those to aquellos
cups to tazas
keys to llaves
few to pocos
both to ambos
nobody to nadie
many to muchos
last to último
less to Menos
first to primero
floor to piso
ten to diez
eleven to once
number to número
twelve to doce
hours to horas
sixteen to dieciséis
fourteen to catorce
fifteen to quince
twenty to veinte
nineteen to diecinueve
thirty to treinta
forty to cuarenta
eighteen to Dieciocho
sum to suma
hundred to cien
seventy to setenta
addition to adición
space to espacio
per to por
meter to metro
average to promedio
give to dar
half to mitad
fourth to cuarto

put to poner
lost to perdió
called to llamado
decided to decidido
died to murió
tried to intentó
knew to sabía
returned to devuelto
happened to sucedió
finished to terminado
opened to abrió
window to ventana
explained to explicado
stopped to detenido
fell to cayó
answered to contestada
talked to habló
enter to entrar
least to menos
ever to nunca
person to persona
youth to juventud
citizen to ciudadano
lady to dama
victim to víctima
type to tipo
version to versión
control to controlar
profile to perfil
categories to categorías
side to lado
problem to problema
turn to giro
young to joven
younger to mas joven
entry to entrada
awards to premios
edge to borde
free to gratis
long to largo
full to completo
hard to difícil
simple to sencillo
fast to rápido
true to cierto
clear to claro
rich to Rico
poor to pobre
sweet to dulce
pure to puro
sure to Por supuesto
alone to solo
spoken to hablado
written to escrito
closed to cerrado
changed to cambiado
presented to presentado
program to programa
li

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [72]:
#Pam's code
# FOR EVERY ROOT WORD (e.g, "be" instead of "is")
# Make csv of translation and calculate a simple metric of how much of a "cognate" it is
trans_cache = {}

newlines=[]

with open(fn, 'r') as f:
    for line in f:
        if len(line)>1:
            if line[0]!='#':
                # process and translate
                pieces = line.split()
                idx = pieces[0]
                word = pieces[1]
                root_token = pieces[2]
                part_of_speech = pieces[3]
                morph_feats = pieces[4]
                dependency_label = pieces[5]
                dependency_edge_head = pieces[6]

                clean_word = root_token.lower() # Only using root token here rather than word
                if '\'' not in clean_word:
                    if clean_word not in trans_cache:
                        time.sleep(0.2)
                        translation_obj = translator.translate(clean_word,src=src,dest=dest)
                        translation = translation_obj.text
                        clean_trans = unidecode(translation).lower() # clean up translation
                        
                        # levenshtein distance features
                        levdist = editdistance.eval(clean_word, clean_trans)
                        levdistfrac = levdist/max(len(clean_word),len(clean_trans))
                        
                        print("%s to %s (LD=%i)" % (clean_word, translation, levdist))
                        trans_cache[clean_word]=translation
                        
                        # word frequency of translation (known language) and src words
                        trans_freq = word_frequency(translation, dest)
                        src_freq = word_frequency(root_token, src)
                        
                        
                        
                        # write to csv file
                        newrow = [root_token, clean_trans, str(src_freq), str(levdist), str(levdistfrac)]
                        newlines.append(",".join(newrow)+'\n')
#                     else:
#                         translation = trans_cache[clean_word]

                
print(len(newlines))
with open('es_en_wordfeats.txt','w') as fp:
    fp.writelines(newlines)

el to he (LD=2)
pan to bread (LD=4)
agua to Water (LD=4)
ser to be (LD=2)
mujere to woman (LD=6)
nosotros to US (LD=7)
beber to to drink (LD=7)
somo to we are (LD=6)
hombre to man (LD=5)
los to the (LD=3)
niño to boy (LD=4)
ellos to they (LD=5)
tú to you (LD=3)
bebe to baby (LD=2)
venir to come (LD=5)
yo to I (LD=2)
bebo to I drink (LD=7)
uno to one (LD=2)
persona to person (LD=1)
hola to Hello (LD=3)
gracia to Grace (LD=2)
bueno to good (LD=5)
día to day (LD=2)
adiós to goodbye (LD=6)
perdón to sorry (LD=5)
por to by (LD=3)
favor to favor (LD=0)
sentir to feel (LD=5)
no to do not (LD=4)
disculpar to excuse (LD=7)
cocino to I cook (LD=5)
cocinar to Cook (LD=5)
pescado to fish (LD=6)
lo to the (LD=3)
sopa to Soup (LD=2)
él to he (LD=2)
comer to eat (LD=5)
fruto to fruit (LD=2)
queso to cheese (LD=4)
jugo to juice (LD=3)
papa to dad (LD=3)
comida to food (LD=4)
almuerzo to lunch (LD=6)
desayuno to breakfast (LD=8)
azúcar to sugar (LD=4)
sal to Salt (LD=1)
soy to I am (LD=4)
vegetariano t

rato to little while (LD=11)
sótano to basement (LD=6)
reina to Queen (LD=4)
jefe to boss (LD=4)
estar to to be (LD=5)
hacer to do (LD=5)
aunque to although (LD=6)
naranja to orange (LD=4)
nado to I swim (LD=6)
voy to I go (LD=4)
precioso to beautiful (LD=8)
bilingüe to bilingual (LD=3)
popular to popular (LD=0)
colore to colore (LD=0)
diferente to different (LD=2)
público to public (LD=2)
propio to own (LD=5)
hora to hour (LD=2)
pasada to pass (LD=3)
jugar to to play (LD=6)
sé to HE (LD=2)
encontrar to find (LD=8)
estudio to study (LD=3)
peso to weight (LD=5)
pesar to to weigh (LD=7)
soñar to Sound (LD=3)
buscar to search (LD=5)
acepto to I agree (LD=7)
volver to return (LD=6)
comenzar to start (LD=7)
caber to to fit (LD=6)
parecer to seem (LD=5)
acabar to finish (LD=6)
firmar to sign (LD=5)
entrar to get in (LD=5)
importar to to import (LD=5)
decir to say (LD=5)
depender to to depend (LD=5)
empezar to start (LD=6)
que to what (LD=4)
ambos to both of them (LD=11)
poco to little bit (L

cuarenta to forty (LD=6)
mayoría to most (LD=5)
tercero to third (LD=5)
comimo to cumin (LD=3)
llover to to rain (LD=7)
dar to give (LD=4)
explicar to explain (LD=4)
volviste to you came back (LD=11)
dónde to where (LD=4)
pasó to He passed (LD=6)
recibir to to receive (LD=7)
decidir to decide (LD=2)
pedir to ask (LD=5)
mirar to look (LD=5)
ocurrir to occur (LD=4)
siempre to always (LD=7)
contestar to answer (LD=6)
perder to to lose (LD=7)
caer to fall (LD=3)
iniciar to Start (LD=6)
ganar to win (LD=4)
cayó to it fell (LD=7)
permitir to allow (LD=8)
ofrecer to offer (LD=3)
intentar to try (LD=7)
insistir to insist (LD=2)
tratar to try (LD=4)
sacar to take (LD=4)
comprar to to buy (LD=6)
cosa to thing (LD=5)
regreso to he came back (LD=10)
salvo to except (LD=6)
botella to bottle (LD=3)
continuar to continue (LD=2)
vivir to live (LD=3)
utilizar to use (LD=7)
responder to answer (LD=6)
entender to understand (LD=8)
contener to contain (LD=4)
gasto to spending (LD=8)
costar to Cost (LD=2)


vive to lives (LD=2)
pertenecer to belong (LD=8)
repetir to repeat (LD=3)
trabajadore to workers (LD=10)
demostrar to show (LD=8)
bandera to flag (LD=7)
británico to British (LD=4)
está to this (LD=4)
adelante to ahead (LD=5)
perdido to lost (LD=7)
especial to special (LD=1)
superior to higher (LD=7)
profundo to deep (LD=8)
puro to pure (LD=1)
seco to dry (LD=4)
particulare to particulare (LD=0)
elevado to high (LD=7)
escaso to little (LD=6)
previo to previous (LD=2)
débil to weak (LD=5)
algo to something (LD=9)
exterior to Exterior (LD=0)
lado to side (LD=3)
entrada to entry (LD=3)
derecho to law (LD=7)
orientación to orientation (LD=2)
salida to departure (LD=8)
tráfico to traffic (LD=3)
viaje to travel (LD=4)
espero to I hope (LD=6)
llegada to arrival (LD=7)
volar to fly (LD=4)
norte to north (LD=1)
sur to south (LD=3)
rumbo to course (LD=5)
dirección to address (LD=7)
sumo to sumo (LD=0)
educación to education (LD=2)
ejemplo to example (LD=3)
relato to story (LD=6)
problema to prob

pez to fish (LD=4)
verdadero to true (LD=7)
vamo to vamo (LD=0)
veo to I see (LD=4)
miro to He looked (LD=8)
rasuradoras to shavers (LD=7)
televisor to TV (LD=7)
lapicero to pen (LD=6)
borracho to drunk (LD=7)
intoxicar to intoxicate (LD=2)
comés to you eat (LD=6)
sándwich to sandwich (LD=1)
tuyo to of yours (LD=6)
amanecer to dawn (LD=7)
disculpadme to excuse me (LD=7)
tocás to you touch (LD=7)
vimo to vimo (LD=0)
lonche to lunch (LD=2)
ahí to there (LD=4)
hablamo to I speak (LD=7)
guarda to guard (LD=1)
ama to loves (LD=5)
españa to Spain (LD=3)
inglaterra to England (LD=6)
ave to bird (LD=4)
bello to beautiful (LD=7)
yendo to going (LD=5)
.. to .. (LD=0)
responde to answer back (LD=10)
david to David (LD=0)
sofia to Sofia (LD=0)
disculpe to sorry (LD=7)
corro to I run (LD=4)
camarón to shrimp (LD=7)
intentaste to you tried (LD=9)
saltar to skip (LD=5)
actriz to actress (LD=3)
ud.. to You.. (LD=3)
maria to Maria (LD=0)
juan to Juan (LD=0)
mesonero to innkeeper (LD=8)
compro to I buy 

ambulancia to ambulance (LD=2)
virus to virus (LD=0)
socorro to Help (LD=7)
investigación to investigation (LD=2)
carro to car (LD=2)
contrato to contract (LD=2)
propiedad to property (LD=4)
mercado to market (LD=5)
estampilla to stamp (LD=5)
peligro to danger (LD=6)
conciencia to awareness (LD=8)
ritmo to rhythm (LD=4)
carrera to career (LD=2)
sociedad to society (LD=3)
ley to law (LD=2)
ministro to Minister (LD=2)
gobernadora to governor (LD=4)
chaqueta to jacket (LD=5)
vista to view (LD=3)
deber to duty (LD=4)
boca to mouth (LD=4)
cerrada to closed (LD=5)
mosca to fly (LD=5)
disculpáme to excuse me (LD=6)
tomo to I take (LD=5)
entrá to enters (LD=2)
leés to read (LD=3)
bienvenido to you are welcome (LD=13)
refri to Refri (LD=0)
juega to play (LD=5)
1689
